In [1]:
import requests
import urllib.parse
import hashlib
import hmac
import base64
import json
import pandas as pd
import numpy as np
import krakenex
from pykrakenapi import KrakenAPI

from API.Endpoints import KrakenEndpoints
from API import KrakenRequests
from Helper import Utilities as util

In [67]:
api = krakenex.API()
k = KrakenAPI(api, tier="None", crl_sleep = 1)
ohlc, last = k.get_ohlc_data("BCHUSD")
print(ohlc)

                           time    open    high     low   close    vwap  \
dtime                                                                     
2022-04-14 01:41:00  1649900460  344.51  344.51  344.51  344.51  344.51   
2022-04-14 01:40:00  1649900400  344.24  344.29  344.24  344.29  344.27   
2022-04-14 01:39:00  1649900340  344.24  344.24  344.24  344.24    0.00   
2022-04-14 01:38:00  1649900280  344.24  344.24  344.24  344.24    0.00   
2022-04-14 01:37:00  1649900220  344.24  344.24  344.24  344.24    0.00   
...                         ...     ...     ...     ...     ...     ...   
2022-04-13 13:46:00  1649857560  328.97  328.97  328.97  328.97    0.00   
2022-04-13 13:45:00  1649857500  328.93  328.97  328.93  328.97  328.93   
2022-04-13 13:44:00  1649857440  328.67  329.03  328.67  329.03  328.96   
2022-04-13 13:43:00  1649857380  328.75  328.75  328.75  328.75    0.00   
2022-04-13 13:42:00  1649857320  328.50  328.75  328.50  328.75  328.67   

                       v

In [55]:
data = k.get_asset_info()
data

,aclass,altname,decimals,display_decimals
1INCH,currency,1INCH,10,5
AAVE,currency,AAVE,10,5
ACA,currency,ACA,10,5
ADA,currency,ADA,8,6
ADA.S,currency,ADA.S,8,6
...,...,...,...,...
ZEUR,currency,EUR,4,2
ZGBP,currency,GBP,4,2
ZJPY,currency,JPY,2,0
ZRX,currency,ZRX,10,5


In [56]:
pairs = k.get_tradable_asset_pairs()
pairs.T

,1INCHEUR,1INCHUSD,AAVEAUD,AAVEETH,AAVEEUR,AAVEGBP,AAVEUSD,AAVEXBT,ACAEUR,ACAUSD,...,YGGEUR,YGGUSD,ZEURZUSD,ZGBPZUSD,ZRXEUR,ZRXGBP,ZRXUSD,ZRXXBT,ZUSDZCAD,ZUSDZJPY
altname,1INCHEUR,1INCHUSD,AAVEAUD,AAVEETH,AAVEEUR,AAVEGBP,AAVEUSD,AAVEXBT,ACAEUR,ACAUSD,...,YGGEUR,YGGUSD,EURUSD,GBPUSD,ZRXEUR,ZRXGBP,ZRXUSD,ZRXXBT,USDCAD,USDJPY
wsname,1INCH/EUR,1INCH/USD,AAVE/AUD,AAVE/ETH,AAVE/EUR,AAVE/GBP,AAVE/USD,AAVE/XBT,ACA/EUR,ACA/USD,...,YGG/EUR,YGG/USD,EUR/USD,GBP/USD,ZRX/EUR,ZRX/GBP,ZRX/USD,ZRX/XBT,USD/CAD,USD/JPY
aclass_base,currency,currency,currency,currency,currency,currency,currency,currency,currency,currency,...,currency,currency,currency,currency,currency,currency,currency,currency,currency,currency
base,1INCH,1INCH,AAVE,AAVE,AAVE,AAVE,AAVE,AAVE,ACA,ACA,...,YGG,YGG,ZEUR,ZGBP,ZRX,ZRX,ZRX,ZRX,ZUSD,ZUSD
aclass_quote,currency,currency,currency,currency,currency,currency,currency,currency,currency,currency,...,currency,currency,currency,currency,currency,currency,currency,currency,currency,currency
quote,ZEUR,ZUSD,ZAUD,XETH,ZEUR,ZGBP,ZUSD,XXBT,ZEUR,ZUSD,...,ZEUR,ZUSD,ZUSD,ZUSD,ZEUR,ZGBP,ZUSD,XXBT,ZCAD,ZJPY
lot,unit,unit,unit,unit,unit,unit,unit,unit,unit,unit,...,unit,unit,unit,unit,unit,unit,unit,unit,unit,unit
pair_decimals,3,3,2,4,2,2,2,6,3,3,...,3,3,5,5,3,3,3,8,5,3
lot_decimals,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
lot_multiplier,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [57]:
usd_pairs = pairs.loc[pairs["quote"] == "ZUSD"]

In [58]:
usd_pairs

,altname,wsname,aclass_base,base,aclass_quote,quote,lot,pair_decimals,lot_decimals,lot_multiplier,leverage_buy,leverage_sell,fees,fees_maker,fee_volume_currency,margin_call,margin_stop,ordermin
1INCHUSD,1INCHUSD,1INCH/USD,currency,1INCH,currency,ZUSD,unit,3,8,1,[],[],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40,1
AAVEUSD,AAVEUSD,AAVE/USD,currency,AAVE,currency,ZUSD,unit,2,8,1,"[2, 3]","[2, 3]","[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40,0.02
ACAUSD,ACAUSD,ACA/USD,currency,ACA,currency,ZUSD,unit,3,8,1,[],[],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40,0.5
ADAUSD,ADAUSD,ADA/USD,currency,ADA,currency,ZUSD,unit,6,8,1,"[2, 3]","[2, 3]","[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40,2.5
AIRUSD,AIRUSD,AIR/USD,currency,AIR,currency,ZUSD,unit,4,8,1,[],[],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YFIUSD,YFIUSD,YFI/USD,currency,YFI,currency,ZUSD,unit,0,8,1,[],[],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40,0.00015
YGGUSD,YGGUSD,YGG/USD,currency,YGG,currency,ZUSD,unit,3,8,1,[],[],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40,2
ZEURZUSD,EURUSD,EUR/USD,currency,ZEUR,currency,ZUSD,unit,5,8,1,[],[],"[[0, 0.2], [50000, 0.16], [100000, 0.12], [250...","[[0, 0.2], [50000, 0.16], [100000, 0.12], [250...",ZUSD,80,40,5
ZGBPZUSD,GBPUSD,GBP/USD,currency,ZGBP,currency,ZUSD,unit,5,8,1,[],[],"[[0, 0.2], [50000, 0.16], [100000, 0.12], [250...","[[0, 0.2], [50000, 0.16], [100000, 0.12], [250...",ZUSD,80,40,5


In [59]:
ticker = k.get_ticker_information('ZRXUSD')
ticker.T

,ZRXUSD
a,"[0.76200, 3000, 3000.000]"
b,"[0.76000, 8789, 8789.000]"
c,"[0.76100, 48.93631000]"
h,"[0.76800, 0.77600]"
l,"[0.75900, 0.70600]"
o,0.76200
p,"[0.76532, 0.73913]"
t,"[38, 925]"
v,"[12169.53064882, 623807.59375978]"


### grab all the OHLC for each crypto pair

In [70]:
list_df = []
for row in usd_pairs.iterrows():
	_, altname = row
	ohlc = k.get_ohlc_data(row[0], interval = 1440, ascending = True)
	print(row[0])
	list_df.append(ohlc)

1INCHUSD
public call frequency exceeded (seconds=0.193494) 
 sleeping for 1 seconds
AAVEUSD
public call frequency exceeded (seconds=0.133677) 
 sleeping for 1 seconds
ACAUSD
public call frequency exceeded (seconds=0.244582) 
 sleeping for 1 seconds
ADAUSD
public call frequency exceeded (seconds=0.146819) 
 sleeping for 1 seconds
AIRUSD
public call frequency exceeded (seconds=0.165876) 
 sleeping for 1 seconds
AKTUSD
public call frequency exceeded (seconds=0.169606) 
 sleeping for 1 seconds
ALCXUSD
public call frequency exceeded (seconds=0.347276) 
 sleeping for 1 seconds
ALGOUSD
public call frequency exceeded (seconds=0.286103) 
 sleeping for 1 seconds
ALICEUSD
public call frequency exceeded (seconds=0.195808) 
 sleeping for 1 seconds
ANKRUSD
public call frequency exceeded (seconds=0.172467) 
 sleeping for 1 seconds
ANTUSD
public call frequency exceeded (seconds=0.125334) 
 sleeping for 1 seconds
APEUSD
public call frequency exceeded (seconds=0.172766) 
 sleeping for 1 seconds
ASTRUSD


In [71]:
list_df

[(                  time   open   high    low  close   vwap         volume  \
  dtime                                                                      
  2021-08-10  1628553600  2.965  3.121  2.680  2.797  2.772   49961.053530   
  2021-08-11  1628640000  2.797  3.500  2.797  2.969  2.999   74540.913066   
  2021-08-12  1628726400  2.979  3.063  2.714  2.772  2.842   31159.505730   
  2021-08-13  1628812800  2.823  3.076  2.823  3.054  2.983   14623.563758   
  2021-08-14  1628899200  3.084  3.104  2.932  3.050  3.023   13866.226847   
  ...                ...    ...    ...    ...    ...    ...            ...   
  2022-04-10  1649548800  1.657  1.657  1.587  1.587  1.633   41284.523988   
  2022-04-11  1649635200  1.581  1.600  1.423  1.454  1.504   64340.331123   
  2022-04-12  1649721600  1.461  1.525  1.445  1.517  1.497  102849.198550   
  2022-04-13  1649808000  1.517  1.554  1.426  1.551  1.512  102537.553901   
  2022-04-14  1649894400  1.551  1.568  1.548  1.565  1.555   20

In [72]:
len(list_df)

147

In [63]:
ohlc = k.get_ohlc_data('BTCUSD', interval=1440, ascending = True)
ohlc[0].head()

,time,open,high,low,close,vwap,volume,count
dtime,,,,,,,,
2020-04-25,1587772800,7509.4,7731.8,7442.2,7550.1,7557.8,5605.490597,12772
2020-04-26,1587859200,7550.0,7710.0,7480.0,7703.9,7613.5,4518.490227,11633
2020-04-27,1587945600,7706.6,7795.0,7629.7,7792.4,7716.1,6358.904211,15933
2020-04-28,1588032000,7792.4,7794.9,7671.1,7755.8,7729.4,4236.805664,9992
2020-04-29,1588118400,7753.0,8978.0,7727.6,8779.9,8403.2,20471.051907,54201


SMA's

In [17]:
ohlc[0]['20 SMA'] = ohlc[0]['close'].rolling(20).mean()
ohlc[0].tail()

,time,open,high,low,close,vwap,volume,count,20 SMA
dtime,,,,,,,,,
2022-04-09,1649462400,42282.6,42805.1,42111.0,42764.0,42428.0,621.746082,13343,44776.395
2022-04-10,1649548800,42781.7,43416.4,41805.4,42160.0,42649.3,1121.270951,15302,44833.680
2022-04-11,1649635200,42164.9,42411.0,39240.0,39534.6,40768.1,3303.899091,31753,44691.520
2022-04-12,1649721600,39530.5,40683.4,39269.4,40078.7,39996.9,2170.327376,24269,44549.940
2022-04-13,1649808000,40078.7,41548.4,39598.0,41307.0,40736.2,2376.591454,23394,44415.840


In [18]:
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure(data=[go.Candlestick(x = ohlc[0].index,
                                    open = ohlc[0]['open'],
                                    high = ohlc[0]['high'],
                                    low = ohlc[0]['low'],
                                    close = ohlc[0]['close'],
                                    ),
                     go.Scatter(x=ohlc[0].index, y=ohlc[0]['20 SMA'], line=dict(color='purple', width=1))])

fig.show()